# Creating a cobaya-compatible likelihood wrapper 

## Installing cobaya

Simply do:

    pip install cobaya
    
and use cobaya to install cosmology modules (CLASS and CAMB, etc.) in `modules_path` (edit the following line appropriately):

In [1]:
modules_path = '/Users/verag/Research/cosmology/modules'

    cobaya-install cosmo -m <modules_path>
    
Now you're set, and modules_path is where you can find CLASS, CAMB, and the usual cosmological data (Planck, SNe, BAO, etc). If you run into trouble with this step, refer to [cobaya's readthedocs page](https://cobaya.readthedocs.io/en/latest/index.html).
    

## Example 1: ACTPol season-2 

Begin by cloning the [python wrapper to the ACTPol-season-2 likelihood](https://github.com/ACTCollaboration/actpols2_like_py):

    git clone https://github.com/ACTCollaboration/actpols2_like_py

and defining a path to it (editing the following line appropriately):

In [2]:
act_repo_path = '/Users/verag/Research/Repositories/ACT_public/actpols2_like_py'

You're now ready to define an instance of an `ACTPol_s2` object that has a `.loglike` method (returns log-likelihood, given path to ACT data, located in ACT repository):

In [3]:
import sys
import os

sys.path.append(act_repo_path)
act_data_path = os.path.join(act_repo_path, 'data')
import act_like

#choose lmax for this analysis:
l_max = 4000

act = act_like.ACTPol_s2(act_data_path, tt_lmax=l_max)
#NOTE: ACTPol_s2 is the actpol season 2 likelihood.

Here is the cobaya-compatible wrapper for ACTPol_s2, following the [example](https://cobaya.readthedocs.io/en/latest/cosmo_external_likelihood.html) in the cobaya documentation:

In [4]:
def actpols2_like(yp=0.95, _theory={'Cl': {'tt': l_max, 'te': l_max, 'ee': l_max}}, _derived={}):
    Cls = _theory.get_cl(ell_factor=True)
    return act.loglike(Cls['tt'][2: l_max+1], 
                       Cls['te'][2: l_max+1], 
                       Cls['ee'][2: l_max+1], yp)

To evaluate this likelihood through cobaya, you need to run cobaya with appropriate input. This is where you tell cobaya to use your ACT wrapper as the likelihood function, and also where you specify all the sampling and fixed parameters. Cobaya takes all this information in the form of a nested dictionary; here is an example:

In [6]:
info = {
        # 'actpols2' is an arbitrary name; this is where you point to the ACT wrapper.
        'likelihood': {'actpols2': actpols2_like},
    
        # modules_path is where you installed the cobaya modules.
        'modules': modules_path,
        
        # these are cosmological and experimental parameters.
        # if prior is specified, parameter is sampled; otherwise, it's fixed.
        'params': {'As': {'latex': 'A_s', 'prior': {'max': 4e-09, 'min': 1e-09}},
                    'ns': {'latex': 'n_s', 'prior': {'max': 1.1, 'min': 0.9}},
                    'H0': 68,
                    'mnu': 0.06,
                    'nnu': 3.046,
                    'num_massive_neutrinos': 1,
                    'ombh2': 0.022,
                    'omch2': 0.12,
                    'tau': 0.07,
                    'yp': 0.95},
    
        # choose the theory code and their parameters: camb or class.
        'theory': {'camb': {'stop_at_error': True}},
        
        # specify the sampler and its parameters, if you want to run chains.
        'sampler': {'mcmc': None},
    
        # this is where the chains would go if you were running a sampler.
        'output': 'chains/actpol_s2',
       }

Initialize an instance of the cobaya [`Model` object](https://cobaya.readthedocs.io/en/latest/cosmo_model.html):

In [7]:
from cobaya.model import get_model
model = get_model(info)

[model] *WARNING* Ignored blocks/options: ['sampler', 'output']
[camb] Importing *local* CAMB from /Users/verag/Research/cosmology/modules/code/CAMB


Finally, evaluate the likelihood at an arbitrary point in cosmological parameter space (the arguments must be the same as the sampling parameters defined above):

In [8]:
model.loglike({'As': 3e-9, 'ns':0.96}, cached=False)

(298.12980991354993, [])